In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

# S5
### Para los negocios que indican su horario, calcule la probabilidad de que un negocio esté abierto a las 21 horas del domingo. Si el domingo no apareciera en el diccionario asuma que no abre.

In [ ]:
import json

In [ ]:
businessesDf = pd.read_csv('/content/drive/MyDrive/Orga Datos/TP2/Dataset/business.csv')
businessesDf.to_parquet('/content/drive/MyDrive/Orga Datos/TP2/business.parquet')

In [ ]:
businessesDf = sqlContext.read.parquet('/content/drive/MyDrive/Orga Datos/TP2/business.parquet', header=True, inferSchema=True)
businessesDfRdd = businessesDf.rdd

In [ ]:
businessesHours = businessesDfRdd.map(lambda x: (x.name, x.hours))\
                    .filter(lambda x: x[1])\
                    .map(lambda x: (x[0], json.loads(x[1].replace("'", '"'))))

In [ ]:
numberOfBusinessesOpenSundayAt21 = businessesHours.filter(lambda x: 'Sunday' in x[1].keys())\
               .filter(lambda x: int(x[1]['Sunday'].split('-')[1].split(':')[0]) >= 21)\
               .count()

In [ ]:
numberOfBusinesses = businessesHours.count()

In [ ]:
probability = numberOfBusinessesOpenSundayAt21 / numberOfBusinesses

In [ ]:
print("La probabilidad de que un negocio este abierto un Domingo a las 21 es:", probability)

La probabilidad de que un negocio este abierto un Domingo a las 21 es: 0.20529723181485648


# S17
### La antigüedad promedio de los usuarios y el nombre del usuario más antiguo cuyas última review contenga la palabra ‘pizza’

In [ ]:
reviewsDf = pd.read_csv('/content/drive/MyDrive/Orga Datos/TP2/Dataset/review.csv')
usersDf = pd.read_csv('/content/drive/MyDrive/Orga Datos/TP2/Dataset/user.csv')

In [ ]:
reviewsDfSample = reviewsDf[:100000]
usersDfSample = usersDf[:100000]

In [ ]:
reviewsDfSample.to_parquet('/content/drive/MyDrive/Orga Datos/TP2/review.parquet')
usersDfSample.to_parquet('/content/drive/MyDrive/Orga Datos/TP2/user.parquet')

In [ ]:
# Reviews
reviewsPar = sqlContext.read.parquet('/content/drive/MyDrive/Orga Datos/TP2/review.parquet', header=True, inferSchema=True)
reviewsParRdd = reviewsPar.rdd
# Users
usersPar = sqlContext.read.parquet('/content/drive/MyDrive/Orga Datos/TP2/user.parquet', header=True, inferSchema=True)
usersParRdd = usersPar.rdd

In [ ]:
# Primer parte del ejercicio

In [ ]:
usersAntiquity = usersParRdd.map(lambda x: (x.user_id, (x.yelping_since).split(' ')[0]))\
                            .filter(lambda x: x[1])

In [ ]:
usersAntiquityYears = usersAntiquity.map(lambda x: (x[0], x[1].split('-')))\
                                    .map(lambda x: int(x[1][0]))

In [ ]:
yearSum = usersAntiquityYears.reduce(lambda x,y: x + y)

In [ ]:
totalUsers = usersAntiquity.count()

In [ ]:
print('La antiguedad promedio en anios es:', yearSum//totalUsers)

La antiguedad promedio en anios es: 2011


In [ ]:
# Segunda parte mismo ejercicio

In [ ]:
usersReviews = reviewsParRdd.map(lambda x: (x.user_id, x.text)).filter(lambda x: x[1])

In [ ]:
usersPizzaReviews = usersReviews.filter(lambda x: 'pizza' in x[1])

In [ ]:
usersPizzaReviewsAndDates = usersAntiquity.join(usersPizzaReviews)

In [ ]:
userIdPizzaReviewDate = usersPizzaReviewsAndDates.map(lambda x: (x[0], x[1][0]))

In [ ]:
userNames = usersParRdd.map(lambda x: (x.user_id, x.name))

In [ ]:
oldestUsers = userIdPizzaReviewDate.join(userNames)

In [ ]:
oldestUsersName = oldestUsers.map(lambda x: (x[1][0], x[1][1])).reduce(lambda x,y: x if x < y else y)

In [ ]:
print("El usuario mas antiguo con la palabra pizza en una de sus reviews es:", oldestUsersName[1])

El usuario mas antiguo con la palabra pizza en una de sus reviews es: Mel


# S18
### Queremos saber cuáles son los negocios más sexys, para esto vamos a ver cuáles son los nombres de los 3 negocios con más tips que contengan el texto “sexy”

In [ ]:
tipsDf = pd.read_csv('/content/drive/MyDrive/Orga Datos/TP2/Dataset/tip.csv')

In [ ]:
tipsDf.to_parquet('/content/drive/MyDrive/Orga Datos/TP2/tip.parquet')

In [ ]:
# Reviews
tipsDf = sqlContext.read.parquet('/content/drive/MyDrive/Orga Datos/TP2/tip.parquet', header=True, inferSchema=True)
tipsDfRdd = tipsDf.rdd

In [ ]:
businessesTips = tipsDfRdd.map(lambda x: (x.business_id, x.text)).filter(lambda x: x[1])

In [ ]:
businessesSexyTips = businessesTips.filter(lambda x: 'sexy' in x[1])

In [ ]:
amountBusinessesSexyTips = businessesSexyTips.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)

In [ ]:
businessesName = businessesDfRdd.map(lambda x: (x.business_id, x.name))

In [ ]:
sexybusinesses = businessesName.join(amountBusinessesSexyTips)

In [ ]:
topThreeSexys = sexybusinesses.takeOrdered(3, key=lambda x: -x[1][1])

In [ ]:
print("Los tres negocios mas sexys son:", topThreeSexys[0][1][0], ",", topThreeSexys[1][1][0], "y", topThreeSexys[2][1][0])

Los tres negocios mas sexys son: The Butcher And Barkeep , Mojo Tapas y Herosame


# S31
### Para los usuarios que tengan más de 100 reviews registradas, ¿cuál es el user_id cuya mediana del tiempo entre reviews es menor?

In [ ]:
import statistics as stats
from datetime import datetime

In [ ]:
def timeBetweenReviewsFunc(dates):
  dates.sort(reverse=True)
  timeBetween = []
  for i in range(len(dates)):
    if i == 0: 
      y = dates[i]
    else:
      timeBetween.append((y-dates[i]).days)
      y = dates[i]
  return timeBetween

In [ ]:
usersDf = pd.read_csv('/content/drive/MyDrive/Orga Datos/TP2/Dataset/user.csv')
reviewsDf = pd.read_csv('/content/drive/MyDrive/Orga Datos/TP2/Dataset/review.csv')

In [ ]:
sampleUsers = usersDf[:100000]
sampleReviews = reviewsDf[:100000]

In [ ]:
sampleUsers.to_parquet('/content/drive/MyDrive/Orga Datos/TP2/user.parquet')
sampleReviews.to_parquet('/content/drive/MyDrive/Orga Datos/TP2/review.parquet')

In [ ]:
# Users
usersPar = sqlContext.read.parquet('/content/drive/MyDrive/Orga Datos/TP2/user.parquet', header=True, inferSchema=True)
usersParRdd = usersPar.rdd
# Reviews
reviewsPar = sqlContext.read.parquet('/content/drive/MyDrive/Orga Datos/TP2/review.parquet', header=True, inferSchema=True)
reviewsParRdd = reviewsPar.rdd

In [ ]:
# Aca empieza

In [ ]:
moreThan100Reviews = usersParRdd.map(lambda x: (x.user_id, x.review_count)).filter(lambda x: x[1] > 100)

In [ ]:
usersReviewsDate = reviewsParRdd.map(lambda x: (x.user_id, (x.date).split(' ')[0].split('-')))\
                                .map(lambda x: (x[0], [int(y) for y in x[1]]))\
                                .map(lambda x: (x[0], datetime(x[1][0], x[1][1], x[1][2])))

In [ ]:
users100ReviewsDates = moreThan100Reviews.join(usersReviewsDate)
users100ReviewsDates = users100ReviewsDates.map(lambda x: (x[0], [x[1][1]]))

In [ ]:
users100ReviewsDatesGrouped = users100ReviewsDates.reduceByKey(lambda x,y: x + y)

In [ ]:
timeBetweenReviews = users100ReviewsDatesGrouped.map(lambda x: (x[0], timeBetweenReviewsFunc(x[1])))

In [ ]:
timeBetweenReviewsMedian = timeBetweenReviews.filter(lambda x: x[1]).map(lambda x: (x[0], stats.median(x[1])))

In [ ]:
smallestMedianBetweenReviews = timeBetweenReviewsMedian.reduce(lambda x,y: x if x < y else y)

In [ ]:
print("De los usuarios con mas de 100 reviews, el user_id cuya mediana del tiempo entre reviews es menor es:", smallestMedianBetweenReviews[0])

De los usuarios con mas de 100 reviews, el user_id cuya mediana del tiempo entre reviews es menor es: -FxsSuwDbIII7yo5BjHpiA
